<a href="https://colab.research.google.com/github/bee-llel/Machine-Learning/blob/master/ML_from_scratch/Naives_Bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
np.random.seed(42)

In [ ]:
df = pd.read_csv('messages.txt', delimiter='\t', names=['type', 'message'])

## 1) Divide the data into two groups: training and testing
## 2) Parse both the training and test examples to generate both the spam and ham data **sets**

In [ ]:
df.head(100)
for i in range(len(df['type'])):
  if df['type'].iloc[i] == 'ham':
    df['type'].loc[i] = 0
  else:
    df['type'].loc[i] = 1

In [ ]:
#Creating a mask to separate train and test from the dataset. 
msk = np.random.rand(len(df)) < 0.7
X_train = df[msk]
X_test= df[~msk] 

N = 3000
I = len(X_train)

# divide ham_train,spam_train,ham_test,ham_train
X_train_ham = X_train[X_train['type'] == 0]
X_train_spam = X_train[X_train['type'] == 1]
X_test_ham = X_test[X_test['type'] == 0]
X_test_spam = X_test[X_test['type'] == 1]

# X_train = X_train.to_numpy()
# X_test = X_test.to_numpy()
# X_test_spam = X_test_spam.to_numpy()
# X_test_ham = X_test_ham.to_numpy()
# X_train_spam = X_train_spam.to_numpy()
# X_train_ham = X_train_ham.to_numpy()
# data = df.to_numpy()

## 3) Generate a dictionary from the training data

In [ ]:
def make_Dictionary(train_dir,N):
    emails = train_dir['message']
    all_words = []
    for i,line in enumerate(emails):
      words = line.split()
      all_words += words

    dictionary = Counter(all_words)
    for item in list(dictionary): 
        if item.isalpha() == False:
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(N)
    return dictionary

In [ ]:
dictionary_X_train = make_Dictionary(X_train,N)
dictionary_X_train_spam = make_Dictionary(X_train_spam,N)
dictionary_X_train_ham = make_Dictionary(X_train_ham,N)

dictionary_X_test = make_Dictionary(X_test,N)

print(dictionary_X_train)

[('to', 1335), ('you', 1053), ('the', 759), ('and', 541), ('is', 515), ('in', 508), ('for', 412), ('my', 408), ('of', 381), ('your', 353), ('me', 338), ('on', 295), ('have', 278), ('that', 264), ('are', 261), ('it', 249), ('call', 237), ('at', 225), ('with', 223), ('or', 222), ('get', 220), ('not', 219), ('be', 219), ('will', 197), ('so', 196), ('ur', 190), ('can', 188), ('but', 183), ('You', 172), ('if', 166), ('from', 162), ('just', 162), ('do', 159), ('know', 154), ('when', 151), ('up', 148), ('like', 148), ('we', 147), ('out', 145), ('go', 145), ('this', 136), ('was', 136), ('all', 134), ('now', 130), ('come', 129), ('am', 129), ('got', 120), ('time', 103), ('about', 101), ('by', 98), ('only', 96), ('send', 94), ('then', 94), ('want', 93), ('one', 91), ('Call', 91), ('If', 90), ('still', 88), ('But', 87), ('How', 83), ('what', 82), ('need', 82), ('as', 82), ('its', 81), ('No', 80), ('going', 80), ('text', 79), ('Just', 77), ('our', 76), ('So', 76), ('he', 75), ('no', 75), ('good', 

## 4) Extract features from both the training data and test data

In [ ]:
def extract_features(dff,dictionary,N):
    features_matrix = np.zeros((len(dff), N))
    docID = 0
    emails = dff['message'] #colonne message de X_train
    all_words = []
    for i,line in enumerate(emails): 
        words = line.split()
        for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
                if d[0] == word:
                    wordID = i
                    features_matrix[docID, wordID] = words.count(word)
        docID = docID + 1
    return features_matrix

In [ ]:
train_matrix = extract_features(X_train,dictionary_X_train,N)
test_matrix = extract_features(X_test,dictionary_X_test,N)

In [ ]:
print(train_matrix)
sum_train_matrix = np.sum(train_matrix, axis=1)
print(sum_train_matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [1. 4. 2. ... 0. 0. 0.]
 [4. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[ 2. 53. 37. ...  7. 13.  1.]


## 5) Implement the Naïve Bayes from scratch, and fit it to the training data

In [ ]:
# phi_n_sachant_y_egal_1 
# print(train_matrix)
# print(X_train.head)

def presence_mot(train_matrix,X):
  mot_spam = np.zeros(N)
  mot_ham = np.zeros(N)
  for n in range(0,N):
    count = 0
    count2 = 0
    for i in range(0,I):
        #si le n iè mot est présent dans le i ème message
        #si le message est un spam
          if ((train_matrix[i][n] > 0) & X['type'].iloc[i]) == True :
            count = 1
          elif ((train_matrix[i][n] > 0) &  (not X['type'].iloc[i])) == True :
            count2 = 1
    mot_spam[n] = count
    mot_ham[n] = count2
  return (mot_spam,mot_ham)

# nombre de spam dans train
def nombre_de_spam_ham(X):
  spam = 0
  ham = 0
  for i in range(len(X)):
    if X['type'].iloc[i] == 1:
      spam = spam + 1
    else:
      ham = ham + 1

  return (spam,ham)



In [ ]:
# loooong a calculer
mot_spam,mot_ham = presence_mot(train_matrix,X_train)

In [ ]:
nbre_spam,nbre_ham = nombre_de_spam_ham(X_train)
print(nbre_spam,nbre_ham)

470 3059


In [ ]:
#Parameter
alpha = 0.00000000000000001

In [ ]:
# fit
phi_n_y1 = (mot_spam + alpha)/(nbre_spam + 2*alpha) 
print(phi_n_y1)
phi_n_y0 = (mot_ham + alpha)/(nbre_ham + 2*alpha)
print(phi_n_y0)
phi_y = (nbre_spam + alpha) / ((nbre_spam +nbre_ham) + 2*alpha)
print("phi_y:\n",phi_y)

[0.00212766 0.00212766 0.00212766 ... 0.00212766 0.00212766 0.00212766]
[3.26904217e-04 3.26904217e-04 3.26904217e-04 ... 3.26904217e-21
 3.26904217e-21 3.26904217e-21]
phi_y:
 0.13318220459053556


In [ ]:
# (X_train['type'].iloc[8] and train_matrix[1][1]) == True
# train_matrix[1][1]
# X_train['type'].iloc[8]
# (4>1 and (not 0))

test_matrix
test_matrix[2][4]


0.0

## 6) make a Prediction with test data

In [ ]:
def predict(X,matrix_x,phi_y,phi_n_y1,phi_n_y0, alpha):


  nbre_spam,nbre_ham = nombre_de_spam_ham(X) 
  # laplace smoothing to find a value to replace zero, it is wrong but i dont have
  proba_minimal_Y1 = alpha/(nbre_spam + 2*alpha)
  proba_minimal_Y0 = alpha/(nbre_ham + 2*alpha)
  
  #on initialise toute les proba a 0
  P_X_sachant_Y1 = np.zeros(len(X['message']))
  P_X_sachant_Y0 = np.zeros(len(X['message']))

  for i in range (len(X['message'])):
  #for chaque mot n dans i mail, on recup phi_n_y1 pour le mot n, je multiplie ces proba entr elle P( x | y = 1)
    ####################################
    produit = 1
    for n in range(N):
      if matrix_x[i][n] > 0:
        produit = produit * phi_n_y1[n]

    if produit != 1:
        P_X_sachant_Y1[i] = produit

    ####################################
  #for chaque mot n dans i mail, on recup phi_n_y0 pour le mot n, je multiplie ces proba entr elle P( x | y = 0)
    ####################################
    produit = 1
    for n in range(N):
      if (matrix_x[i][n] > 0):
        produit =  produit * phi_n_y0[n]
     
    if produit != 1:
        P_X_sachant_Y0[i] = produit
    ####################################

    if P_X_sachant_Y1[i] == 0:
      P_X_sachant_Y1[i] = proba_minimal_Y1
    if P_X_sachant_Y0[0] == 0:
      P_X_sachant_Y0[i] = proba_minimal_Y0

  # formule 30 P( y = 0 | x ) et P( y = 1 | x)
  P_Y1_sachant_X =   (P_X_sachant_Y1 * phi_y)/ (P_X_sachant_Y1 * phi_y + P_X_sachant_Y0 * (1 - phi_y))
  P_Y0_sachant_X =   (P_X_sachant_Y0 * (1 - phi_y)) / (P_X_sachant_Y1 * phi_y + P_X_sachant_Y0 * (1 - phi_y))

  #formule 28 pour faire la prediction argmax y P(x|y), on compare les deux listes terme à terme 
  prediction = P_X_sachant_Y0 = np.zeros(len(X['message']))
  for i in range(len(X['message'])):
    if P_Y1_sachant_X[i] > P_Y0_sachant_X[i]:
      prediction[i] = 1
    else:
      prediction[i] = 0

  return prediction
  
  

In [ ]:
prediction =predict(X_test,test_matrix,phi_y,phi_n_y1,phi_n_y0, alpha)

In [ ]:
print(prediction)

[0. 1. 0. ... 0. 0. 0.]


## 7)Measure the spam-filtering performance for each approach through the confusion matrix, precision, and recall.


In [ ]:
accuracy = 0

for i in range(len(prediction)):
  if prediction[i] == (X_test['type'].iloc[i]):
    accuracy += 1
print("accuracy",accuracy/len(prediction))

accuracy 0.6732471068754254


In [ ]:
# Confusion matrix
accuracy = 0
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(len(prediction)):
  if prediction[i] == 0:
    if X_test['type'].iloc[i] == 0:
      TN = TN +1
    else:
      FN = FN + 1
    
  if prediction[i] == 1:
    if X_test['type'].iloc[i] == 0:
      FP = FP +1
    else:
      TP = TP +1
print(TP,FP,TN,FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(precision,recall)

19 297 970 183
0.060126582278481014 0.09405940594059406


## 8. Discuss your results.

The confusion result seem to be broken, the rest is working.

Our prediction function need to be improved, the laplace smoothing is not done properly or can at least be improved.

a very small alpha allow us to reach a high accuracy of 65 percent. a big alpha gives us really bad result

We have created to much variable that are not useful.




